# Used Appliances Query - Jupyter Notebook
This notebook queries the Google API to find 'used appliances near me' within a 90 mile radius of the zip code 29601. The results are saved to a Pandas DataFrame and then to a local CSV file.

In [7]:
# Configuration Block
# Import required libraries
import time
import pandas as pd
import googlemaps
import requests
import calendar
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Google API key
load_dotenv()
API_KEY = os.getenv('API_KEY')

# Initialize Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

In [322]:
# Set location for centering search (using zip code)
location = '29601'

# Set location for adding distance & drive time
driving_origin_location = '420 N Pleasantburg Dr, Greenville, SC 29607'

# Set base radius (90 miles) * some var multiplier to bring it closer/further
radius = 144000 * (2/3)  # 1 mile = 1609.34 meters

# Set query to use for selecting businesses which will serve as the core for the result set
query = 'used and dented appliances near me'



In [339]:
def save_to_csv(dataframe):
    # Get local system time for naming the file
    current_time = time.localtime()
    file_name = f"{current_time.tm_hour}-{current_time.tm_min}.csv"
    
    # Save DataFrame to CSV
    dataframe.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}")


In [324]:
# Enrich a given place with details
def query_google_api_details(place_id):
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        'place_id': place_id,
        'fields': 'name,opening_hours,website,formatted_phone_number',
        'key': API_KEY,
    }
    response = requests.get(base_url, params=params)
    result = response.json()
    return result.get('result', {})


In [325]:
# Entrypoint search call to select places
# Exhaust pagination if necessary
# Enrich results with place details
def query_google_api_places_base(location, radius, query):
    results = []
    # Initial request
    result = gmaps.places(query, location=location, radius=radius)
    results.extend(result['results'])

    # Paginate if there are more results
    while 'next_page_token' in result:
        time.sleep(2)  # Add a short delay as per API guidelines
        result = gmaps.places(query, location=location, radius=radius, page_token=result['next_page_token'])
        results.extend(result['results'])

    # Fetch additional details for each place using query_google_api_details
    detailed_results = []
    for place in results:
        place_id = place.get('place_id')
        if place_id:
            details = query_google_api_details(place_id)
            place.update(details)
            detailed_results.append(place)

    return detailed_results

In [326]:
# Flatten json nested properties following a heirarchical concatenation pattern (Similar to Grafana Loki)
# https://stackoverflow.com/a/57334325
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [327]:
from datetime import datetime

# Explode the opening hours into a map keyed by text representation of the day name with properties for open, open time and close time
def process_opening_hours(opening_hours):
    operating_schedule = {}

    if 'periods' in opening_hours:
        for period in opening_hours['periods']:

            # https://developers.google.com/maps/documentation/places/web-service/details#PlaceOpeningHoursPeriod-close
            # Clients can rely on always-open being represented as an open period containing day with value 0 and time with value 0000, and no close. 
            always_open = period['open']['day'] == 0 and period['open']['time'] == '0000' and 'close' not in period
            if always_open:
                for i in range(0,7):
                    day_name = calendar.day_name[i].lower()
                    operating_schedule[day_name] = {
                        'open': True,
                        'hours_open': '00:00:00',
                        'hours_close': '23:59:59'
                    }
                continue
            
            day_number = period['close']['day']  # The day number (0 for Sunday, 1 for Monday, etc.)
            day_name = calendar.day_name[(day_number+6)%7].lower() # calendar.day_name uses 6 for Sunday, 0 for Monday, etc.

            if period['close']['time'].lower() == 'closed':
                operating_schedule[day_name] = {
                    'open': False,
                    'hours_open': None,
                    'hours_close': None
                }
            else:
                hours_open = datetime.strptime(period['open']['time'], '%H%M').strftime('%H:%M:%S')
                hours_close = datetime.strptime(period['close']['time'], '%H%M').strftime('%H:%M:%S')
                operating_schedule[day_name] = {
                    'open': True,
                    'hours_open': hours_open,
                    'hours_close': hours_close
                }

    return operating_schedule

In [341]:
import time

def save_to_csv(dataframe):
    # Get local system time for naming the file
    current_time = time.localtime()
    
    # Format the file name with year, month, day, hour, and minute
    file_name = f"{current_time.tm_year}-{current_time.tm_mon}-{current_time.tm_mday}_{current_time.tm_hour}-{current_time.tm_min}.csv"
    
    # Save DataFrame to CSV
    dataframe.to_csv(file_name, index=False)
    print(f"Data saved to {file_name}")


In [329]:
import json

# Dump a given result for inspecting shape
def dump_for_inspection(record, name):
    # Specify the file path where you want to save the JSON data
    file_path = name + '.json'
    # Write the result to the JSON file
    with open(file_path, 'w') as json_file:
        json.dump(record, json_file, indent=2)

In [330]:
# Query Google API
results = query_google_api_places_base(location, radius, query)

In [331]:
# Massage the dates into something more human usable
for result in results:
    opening_hours = result.get('opening_hours', {})
    operating_schedule = process_opening_hours(opening_hours)
    result['operating_schedule'] = operating_schedule

In [332]:
# Remove any unwanted properties  
for result in results:
    properties_to_delete = ['plus_code', 'icon_mask_base_uri', 'geometry','photos','opening_hours','types','icon','reference','icon_background_color']
    # Remove specified properties if they exist in the result dictionary
    for prop in properties_to_delete:
        result.pop(prop, None)
# Dump a sample record of the current shape
dump_for_inspection(results[0], 'raw')

In [333]:
# Pull down nested properties as desired, in this case i.e. operating_schedule
for result in results:
    if 'operating_schedule' in result:
        # Extract the operating_schedule from the result
        operating_schedule = result['operating_schedule']

        # Remove the 'operating_schedule' key from the result
        result.pop('operating_schedule', None)

        # Add the individual days with their schedules to the result
        for day, schedule in operating_schedule.items():
            result[day] = schedule


dump_for_inspection(results[0], 'formatted')

In [334]:
# Flatten whatever remains
flattened_results = list(map(flatten_json, results))
# Dump a sample record of the current shape
dump_for_inspection(flattened_results[0], 'flat')

In [335]:
# Convert results to a DataFrame
df = pd.DataFrame(flattened_results)
df.head(5)

,business_status,formatted_address,name,place_id,rating,user_ratings_total,formatted_phone_number,website,sunday_open,sunday_hours_open,...,thursday_open,thursday_hours_open,thursday_hours_close,friday_open,friday_hours_open,friday_hours_close,saturday_open,saturday_hours_open,saturday_hours_close,price_level
0,OPERATIONAL,"605 Haywood Rd, Greenville, SC 29607, United S...",Conn's HomePlus,ChIJ17j05fsuWIgRshyWFuhz-jg,4.5,1272,(864) 675-1331,https://www.conns.com/sc/greenville/605-haywoo...,True,10:00:00,...,True,10:00:00,21:30:00,True,10:00:00,21:30:00,True,09:00:00,21:30:00,NaN
1,OPERATIONAL,"740 Wade Hampton Blvd, Greenville, SC 29609, U...",Endtime Harvest Thrift Store,ChIJ9erFnxcuWIgRLqd6-LHesYo,4.3,45,(864) 271-2505,NaN,NaN,NaN,...,True,09:30:00,16:30:00,True,09:30:00,16:30:00,NaN,NaN,NaN,NaN
2,OPERATIONAL,"620 Laurens Rd, Greenville, SC 29607, United S...",Banks Appliance Parts & Service,ChIJbW-3gzEuWIgRxkf7QUg7XV0,4.2,260,(864) 233-1999,https://www.banksappliance.com/,NaN,NaN,...,True,08:00:00,17:00:00,True,08:00:00,17:00:00,NaN,NaN,NaN,NaN
3,OPERATIONAL,"4007 Old Buncombe Rd, Greenville, SC 29617, Un...",Guaranteed Goods Appliance Sales & Service,ChIJ7f_2UdUzWIgRjxS8oh4WeZA,2.9,74,(864) 516-9146,NaN,NaN,NaN,...,True,10:30:00,18:00:00,True,10:30:00,15:00:00,True,10:30:00,16:00:00,NaN
4,OPERATIONAL,"20 Maple Creek Cir suite a, Greenville, SC 296...",KWR Appliances,ChIJZ_pgGM0nWIgRemTa3vrnPDI,3.8,207,(864) 502-8735,http://www.kwrappliances.com/,True,13:00:00,...,True,10:00:00,18:00:00,True,10:00:00,18:00:00,True,10:00:00,17:00:00,NaN


In [336]:
import concurrent.futures

# Function to calculate driving distance and duration using Google Maps Distance Matrix API
def calculate_driving_info(row):
    destination_address = row[1]  # Use integer index for 'formatted_address'
    origin_address = driving_origin_location

    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        'origins': origin_address,
        'destinations': destination_address,
        'mode': 'driving',
        'key': API_KEY,
    }
    response = requests.get(base_url, params=params)
    result = response.json()

    # Extract distance in meters and duration in minutes from the API response
    try:
        distance_meters = result['rows'][0]['elements'][0]['distance']['value']
        duration_minutes = result['rows'][0]['elements'][0]['duration']['text']
    except (KeyError, IndexError, ValueError):
        distance_meters = None
        duration_minutes = None

    return distance_meters, duration_minutes

# Use concurrent.futures to perform concurrent API calls
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(calculate_driving_info, df.itertuples(index=False)))

# Unpack the results and add new columns to the DataFrame
df['driving_distance_meters'], df['driving_duration'] = zip(*results)

# Add a new column for driving_distance_miles
df['driving_distance_miles'] = df['driving_distance_meters'] / 1609.34  # 1 mile = 1609.34 meters

# Display the updated DataFrame
df.head(5)


,business_status,formatted_address,name,place_id,rating,user_ratings_total,formatted_phone_number,website,sunday_open,sunday_hours_open,...,friday_open,friday_hours_open,friday_hours_close,saturday_open,saturday_hours_open,saturday_hours_close,price_level,driving_distance_meters,driving_duration,driving_distance_miles
0,OPERATIONAL,"605 Haywood Rd, Greenville, SC 29607, United S...",Conn's HomePlus,ChIJ17j05fsuWIgRshyWFuhz-jg,4.5,1272,(864) 675-1331,https://www.conns.com/sc/greenville/605-haywoo...,True,10:00:00,...,True,10:00:00,21:30:00,True,09:00:00,21:30:00,NaN,2676,6 mins,1.662793
1,OPERATIONAL,"740 Wade Hampton Blvd, Greenville, SC 29609, U...",Endtime Harvest Thrift Store,ChIJ9erFnxcuWIgRLqd6-LHesYo,4.3,45,(864) 271-2505,NaN,NaN,NaN,...,True,09:30:00,16:30:00,NaN,NaN,NaN,NaN,3525,7 mins,2.190339
2,OPERATIONAL,"620 Laurens Rd, Greenville, SC 29607, United S...",Banks Appliance Parts & Service,ChIJbW-3gzEuWIgRxkf7QUg7XV0,4.2,260,(864) 233-1999,https://www.banksappliance.com/,NaN,NaN,...,True,08:00:00,17:00:00,NaN,NaN,NaN,NaN,2746,5 mins,1.706290
3,OPERATIONAL,"4007 Old Buncombe Rd, Greenville, SC 29617, Un...",Guaranteed Goods Appliance Sales & Service,ChIJ7f_2UdUzWIgRjxS8oh4WeZA,2.9,74,(864) 516-9146,NaN,NaN,NaN,...,True,10:30:00,15:00:00,True,10:30:00,16:00:00,NaN,10941,15 mins,6.798439
4,OPERATIONAL,"20 Maple Creek Cir suite a, Greenville, SC 296...",KWR Appliances,ChIJZ_pgGM0nWIgRemTa3vrnPDI,3.8,207,(864) 502-8735,http://www.kwrappliances.com/,True,13:00:00,...,True,10:00:00,18:00:00,True,10:00:00,17:00:00,NaN,11800,14 mins,7.332198


In [342]:
save_to_csv(df)

Data saved to 2024-1-28_14-20.csv
